In [ ]:
%matplotlib inline
import numpy as np 
import pandas as pd 
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt 
import seaborn as sns
def ignore_warn(*args, **kwargs):
    pass
import warnings
warnings.warn = ignore_warn
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

一、读取数据

In [ ]:
# 读取文件
df=pd.read_excel(r'/Users/luzzzz/Desktop/数据预测/时间序列数据.xlsx')

In [ ]:
# 查看空值情况
(df.isnull().sum() / len(df)).sort_values(ascending=False)[:20]

二、纠偏

In [ ]:
sns.distplot(df['新增确诊'] , fit=norm);
 
#得到均值和标准差
(mu, sigma) = norm.fit(df['新增确诊'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
 
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('confirmed distribution')

In [ ]:
 #画QQ图
fig = plt.figure()
res = stats.probplot(df['新增确诊'], plot=plt)
plt.show();

In [ ]:
sns.distplot(np.log1p(df['新增确诊']) , fit=norm);
 
# 纠偏,对数化处理。
(mu, sigma) = norm.fit(np.log1p(df['新增确诊']))
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

In [ ]:
#纠偏后再化分布图
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('log(confirmed+1) distribution')

In [ ]:
 
#画QQ图
fig = plt.figure()
res = stats.probplot(np.log1p(df['新增确诊']), plot=plt)
plt.show();

In [ ]:
print("数据清洗前的特征变量的数量为:",len(df.columns)-1)
#对目标变量进行对数化纠偏
df["新增确诊"] = np.log1p(df["新增确诊"])
#对数值型特征变量进行对数化处理:1.过滤出数值型变量,2.计算偏度,3.对偏度大于0.75的数据进行对数化纠偏
#1.过滤出数值型变量
numeric_feats = df.dtypes[df.dtypes != "object"].index
#2.计算偏度
skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
#3.对偏度大于0.75的数据进行对数化纠偏
df[skewed_feats] = np.log1p(df[skewed_feats])
#4.对分类型变量进行one-hot编码,分类型变量会被横向展开为多个变量
df = pd.get_dummies(df)
#用均值填充空值
df = df.fillna(df.mean())
 
print("数据清洗后的特征变量的数量为:",len(df.columns)-1)
#创建训练集和测试集
X, y = df.drop(['新增确诊'], axis = 1), df['新增确诊']
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 0)

三、训练模型

In [ ]:
# 利用弹性网络
cv_model = ElasticNetCV(l1_ratio=0.5, eps=1e-3, n_alphas=100, fit_intercept=True, 
                        normalize=True, precompute='auto', max_iter=2000, tol=0.0001, cv=6, 
                        copy_X=True, verbose=0, n_jobs=-1, positive=False, random_state=0)

In [ ]:
# 训练模型              
cv_model.fit(X_train, y_train)

In [ ]:
# 计算最佳迭代次数、alpha和ratio
print('最佳 alpha: %.8f'%cv_model.alpha_)
print('最佳 l1_ratio: %.3f'%cv_model.l1_ratio_)
print('迭代次数 %d'%cv_model.n_iter_)

In [ ]:
# 输出结果
y_train_pred = cv_model.predict(X_train)
y_pred = cv_model.predict(X_test)
print('Train r2 score: ', r2_score(y_train_pred, y_train))
print('Test r2 score: ', r2_score(y_test, y_pred))
train_mse = mean_squared_error(y_train_pred, y_train)
test_mse = mean_squared_error(y_pred, y_test)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
print('Train RMSE: %.4f' % train_rmse)
print('Test RMSE: %.4f' % test_rmse)